# HOWTO - Use the Vantage APIs to create a systematic processing

## Overview
This example notebook will show how to launch a service in systematic mode giving an area of interest (AOI) and a time range.

Steps:

1. Retrieve service's details
2. Prepare the range and the AOI
3. Create the systematic processing and retrieve the jobs information


More information about the available APIs can be found in the [Hal Browser](https://vantage.earthi.world/secure/api/v2.0/).

## Prepare the environment

In [ ]:
import requests
import json

In [ ]:
import getpass

user = getpass.getpass('Username: ')

### The API key can be retrieved from "MyAccount" tab on the platform: https://vantage.earthi.world/app/#/account

In [ ]:
api = getpass.getpass('API Key: ')

In [ ]:
endpoint = 'https://vantage.earthi.world/secure/api/v2.0'

## 1. Retrieve service's details
In this example we'll pick the service StabiliseVideo filtering the services by "stabilise"

In [ ]:
query = { 'sort':'type,name','filter':'stabilise'}
response = requests.get(endpoint + '/services/search/findByFilterOnly', auth=(user,api), params=(query))
response.status_code

In [ ]:
json_data = json.loads(response.text)
for service in json_data['_embedded']['services']:
    print("Name: {0}\nDescription: {1}\nID: {2}\n".format(service['name'], service['description'], service['id']))

In [ ]:
service_id = '11'
response = requests.get(endpoint + '/services/' + service_id, auth=(user,api))
response.status_code

In [ ]:
response.json()

## 2. Prepare the range and the AOI
The platform takes in input WKT AOI.
It's possible to convert LatLong coordinates with the private endpoint /utils/coords2wkt (PUT)

In [ ]:
headers={'Accept':'application/hal+json', 'Content-Type':'application/hal+json'}
raw_data='{"latitude":"-0.118092","longitude":"51.509865","distance":"10000"}'
response = requests.put(endpoint + '/utils/coords2wkt', auth=(user,api), data=raw_data,headers=headers)
response.status_code

In [ ]:
response.text

### Coordinates example (London) : 
POLYGON((-0.1180920000000053 51.599696528411954, -0.2079231556571279 51.509865, -0.1180920000000053 51.42003347158804, -0.0282605540535424 51.509865, -0.1180920000000053 51.599696528411954))

### Select a time range, in the following example (1week):
'productDateStart': '2021-09-08T00:00:00Z'
'productDateEnd': '2021-09-15T23:59:59Z'

## 3. Create the systematic processing and retrieve the jobs information

In [ ]:
headers={'accept':'application/hal+json', 'Content-Type':'application/hal+json'}
raw_data='{"service":{"id":11,"name":"StabiliseVideo","owner":{"role":"ADMIN","name":"sunitha.vallabhaneni","id":10,"_links":{"self":{"href":"https://vantage.earthi.world/secure/api/v2.0/users/10{?projection}","templated":true}}},"type":"PROCESSOR","_links":{"self":{"href":"https://vantage.earthi.world/secure/api/v2.0/services/11"},"service":{"href":"https://vantage.earthi.world/secure/api/v2.0/services/11{?projection}","templated":true},"owner":{"href":"https://vantage.earthi.world/secure/api/v2.0/services/11/owner"},"contextFiles":{"href":"https://vantage.earthi.world/secure/api/v2.0/services/11/contextFiles"}},"access":{"published":true,"publishRequested":false,"currentLevel":"SUPERUSER"},"description":"Performs video stabilisation by aligning each video frame to a central reference frame","dockerTag":"eovep/videoframesalignment","status":"AVAILABLE","licence":"OPEN"},"inputs":{},"systematicParameter":"InputVideo"}'
payload = {'productDateStart': '2021-09-08T00:00:00Z', 'mission': 'sentinel2', 's2ProcessingLevel': 'L1C', 'maxCloudCover': '10', 'aoi': 'POLYGON((-0.1180920000000053 51.599696528411954, -0.2079231556571279 51.509865, -0.1180920000000053 51.42003347158804, -0.0282605540535424 51.509865, -0.1180920000000053 51.599696528411954))', 'productDateEnd': '2021-09-15T23:59:59Z', 'catalogue': 'SATELLITE'}

#activeFire
#headers={'accept':'application/hal+json', 'Content-Type':'application/hal+json'}
#raw_data='{"service":{"id":2,"name":"ActiveFire","owner":{"role":"ADMIN","name":"alexandre.fournot","id":2,"_links":{"self":{"href":"https://vantage.earthi.world/secure/api/v2.0/users/2{?projection}","templated":true}}},"type":"PARALLEL_PROCESSOR","_links":{"self":{"href":"https://vantage.earthi.world/secure/api/v2.0/services/2"},"service":{"href":"https://vantage.earthi.world/secure/api/v2.0/services/2{?projection}","templated":true},"owner":{"href":"https://vantage.earthi.world/secure/api/v2.0/services/2/owner"},"contextFiles":{"href":"https://vantage.earthi.world/secure/api/v2.0/services/2/contextFiles"}},"access":{"published":false,"publishRequested":false,"currentLevel":"SUPERUSER"},"description":"This service creates a false colour composite from a combination of Sentinel-2 visible, NIR and SWIR bands to visualise regions of active fire.","dockerTag":"eovep/activefire","status":"AVAILABLE","licence":"OPEN"},"inputs":{},"systematicParameter":"parallelInputs"}'
#payload = {'productDateStart': '2021-09-08T00:00:00Z', 'mission': 'sentinel2', 's2ProcessingLevel': 'L1C', 'maxCloudCover': '10', 'aoi': 'POLYGON((-0.1180920000000053 51.599696528411954, -0.2079231556571279 51.509865, -0.1180920000000053 51.42003347158804, -0.0282605540535424 51.509865, -0.1180920000000053 51.599696528411954))', 'productDateEnd': '2021-09-15T23:59:59Z', 'catalogue': 'SATELLITE'}
# Comment the following to avoid launching a job 
response = requests.post(endpoint + '/jobConfigs/launchSystematic', auth=(user,api), data=raw_data,headers=headers,params=payload)
response.status_code

### Get the current user's ID

In [ ]:
response = requests.get(endpoint + '/users/current', auth=(user,api))
response.status_code

In [ ]:
response.json()

### Filter the user's systematicProcessings

In [ ]:
userID = '50'
query = { 'sort,filter':'name','owner':'https://vantage.earthi.world/secure/api/v2.0/users/50'}
#response = requests.get(endpoint + '/services/search/findByFilterOnly', auth=(user,api), params=(query))
#search/findByOwner?sort=name&filter=&owner=https://vantage.earthi.world/secure/api/v2.0/users/50
response = requests.get(endpoint + '/systematicProcessings/search/findByOwner', auth=(user,api), params=(query))
response.status_code

In [ ]:
json_data = json.loads(response.text)
for systematicProcessings in json_data['_embedded']['systematicProcessings']:
    print("ID: {0}\nSTATUS: {1}\n".format(systematicProcessings['id'], systematicProcessings['status']))

### Display the systematicProcessing's jobs

In [ ]:
sysID = '5'
response = requests.get(endpoint + '/systematicProcessings/' + sysID, auth=(user,api))
response.status_code

In [ ]:
response.json()

### Parent job

In [ ]:
sysID = '5'
response = requests.get(endpoint + '/systematicProcessings/' + sysID + '/parentJob', auth=(user,api))
response.status_code

In [ ]:
response.json()

### Subjobs

In [ ]:
parentID = '1185'
response = requests.get(endpoint + '/jobs/' + parentID + '/subJobs', auth=(user,api))
response.status_code

In [ ]:
response.json()

### Job details

In [ ]:
jobID = '1186'
response = requests.get(endpoint + '/jobs/' + jobID, auth=(user,api))
response.status_code

In [ ]:
response.json()

### Retrieve the logs

In [ ]:
jobID = '1185'
response = requests.get(endpoint + '/jobs/' + jobID + '/logs', auth=(user,api))
response.status_code

In [ ]:
response.json()